This notebook can be used to train the mean teacher model. Please leave all parameters unchanged except for **supervised_pct**. 

Make sure to add whatever file path you think will be more convinient. In my case, for each model I created a folder, and 3 subfolders, one for the LB, UP and Mean Teacher. If you choose this method, **make sure the folders are created beforehand**. Otherwise, feel free to do whatever. For this notebook, the following things will be saved: 

1. The weights of the student model every 5 epochs (./whatever_name_you_want_epoch_X).
2. Running loss, supervised loss and unsupervised loss during training (every epoch).
3. Accuracy and IOU on train set every 5 epochs.
4. Accuracy and IOU on validation set every 5 epochs.
5. The weight profile of the unsupervised loss during training (i.e. the $\omega$ in $L = L_{S} + \omega_t L_U$).

**If you use this notebook directly from colab, PLEASE make a copy and don't change anything in the original file, cuz then my kernel will crash :(**.

NOTE: **For very low supervised percentages ($<10%$)**, the batch size might need to be increased. In that case, the wait period should also be changed. 

In [33]:
# ONLY PARAMETER THAT SHOULD BE CHANGED. 
supervised_pct = 0.1 # what percent of training is to be labelled
model_save_path = "./M10_30_03_23/M10_MT/"

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
%ls

data_augmentation.py  M1_28_03_23/   __pycache__/              utils.py
data_into_loaders.py  M2_29_03_23/   Train_Mean_Teacher.ipynb
M10_30_03_23/         model_UNet.py  Train_Supervised.ipynb


In [36]:
%cd /content/gdrive/MyDrive/Colab \Notebooks/Applied DL CW

/content/gdrive/MyDrive/Colab Notebooks/Applied DL CW


In [27]:
%ls

data/                 M1_28_03_23/   Train_Mean_Teacher.ipynb
data_augmentation.py  M2_29_03_23/   Train_Supervised.ipynb
data_into_loaders.py  model_UNet.py  utils.py
M10_30_03_23/         __pycache__/


In [37]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
import model_UNet
from data_augmentation import augmentation, colorjiter, invert
import matplotlib.pyplot as plt
from data_into_loaders import get_data, download_data
from utils import dice_loss, wt, update_ema_variables, unsup_loss, evaluate_model

In [38]:
download_data()

  0%|          | 0/791918971 [00:00<?, ?it/s]

  0%|          | 0/19173078 [00:00<?, ?it/s]

In [39]:
# Do NOT change these hyperparams 
img_resize = 64   
val_pct, test_pct = 0.2, 0.1 # Validation and test set %. 

depth, dropout_rate = 3, 0.25  # U-Net params

# Training params
epochs, lr, lr_gamma = 100, 1e-3, 0.9
batch_size = 32

ramp_up, consistency, wait_period = 25, 1.5, int(1000/161 * 1.1) # Teacher contribution params
alpha = 0.999 # Teacher update params
global_step = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


In [40]:
#### Initialisation ####
#create 2 network
modelS = model_UNet.UNet(in_channels=3, num_classes=2, depth=depth)
modelT = model_UNet.UNet(in_channels=3, num_classes=2, depth=depth)
modelS,  modelT= modelS.to(device), modelT.to(device)

#optimizer
optimizer = Adam(modelS.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=lr_gamma, last_epoch=-1, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-03.


/content/gdrive/MyDrive/Colab Notebooks/Applied DL CW/model_UNet.py:211: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
/content/gdrive/MyDrive/Colab Notebooks/Applied DL CW/model_UNet.py:212: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


In [41]:
mixed_train_loader, val_loader, test_loader = get_data(supervised_pct,1 - supervised_pct, val_pct, test_pct, batch_size=batch_size, img_resize=img_resize)

all images are =  7393


In [42]:
# Train
eval_freq = 5
losses, sup_losses, unsup_losses, accsTr, IousTr, accsVal, IousVal, wts = [], [], [], [], [], [], [] , []

modelS.train() # Put model in train mode - important because we have dropout

for epoch in range(epochs):

        running_loss, running_loss_sup, running_loss_unsup = 0, 0, 0 # Init running losses
        w_t = wt(rampup_length=ramp_up, current=epoch, alpha=consistency, wait_period=wait_period) # Get unsupervised weight for the epoch

        for step, data in enumerate(mixed_train_loader):

            imgs, labs = data
            imgS_aug = augmentation(imgs) # Augment images

            imgS_aug = imgS_aug.to(device)
            labs = labs.squeeze().type(torch.LongTensor).to(device)

            optimizer.zero_grad()

            # Forward pass for student and teacher
            z = modelS(imgS_aug) 

            sup_idx = torch.tensor([(elem != -1).item() for elem in labs[:, 0, 0]]).to(device) #If batchsize is the first dim

            Ls = dice_loss(z[sup_idx], labs[sup_idx])
            Lu = unsup_loss(z, modelT, imgs, device)
            loss = Ls + w_t * Lu
            
            loss.backward()
            
            optimizer.step()    
            global_step += 1
            update_ema_variables(modelS, modelT, alpha, global_step)
            running_loss += loss.item()
            running_loss_sup += Ls.item()
            running_loss_unsup += Lu.item()

        print(f'Epoch: {epoch + 1:4d} - Loss: {running_loss:6.2f}, loss_sup: {running_loss_sup:6.1f}, loss_unsup: {running_loss_unsup:6.1f}, w_t: {w_t: 3.2f}')
        losses.append(running_loss)
        sup_losses.append(running_loss_sup)
        unsup_losses.append(running_loss_unsup)
        wts.append(w_t)

        if (epoch % eval_freq == 0):

          # Get accuracy and IOU for train and validation dataset 
          accTr, IouTr = evaluate_model(modelS, mixed_train_loader, device)
          accVal, IouVal = evaluate_model(modelS, val_loader, device)

          accsTr.append(accTr)
          IousTr.append(IouTr)   
          accsVal.append(accVal)
          IousVal.append(IouVal)

          print(f'For training: accuracy-{accTr:2.0%}; IOU-{IouTr:2.0%}')
          print(f'For validation: accuracy-{accVal:2.0%}; IOU-{IouVal:2.0%}')

          np.savetxt(f"{model_save_path}running_loss", losses)
          np.savetxt(f"{model_save_path}sup_loss", sup_losses)
          np.savetxt(f"{model_save_path}unsup_loss", unsup_losses)

          np.savetxt(f"{model_save_path}train_acc", accsTr)
          np.savetxt(f"{model_save_path}train_IOU", IousTr)
          np.savetxt(f"{model_save_path}val_acc", accsVal)
          np.savetxt(f"{model_save_path}val_IOU", IousVal)

          np.savetxt(f"{model_save_path}weights", wts)

          torch.save(modelS.state_dict(), f"{model_save_path}student_epoch_{epoch+1}" + '.pt')

        #if (epoch+1)%8 == 0:
        #    scheduler.step()



/content/gdrive/MyDrive/Colab Notebooks/Applied DL CW/utils.py:47: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


Epoch:    1 - Loss:  43.73, loss_sup:   43.7, loss_unsup:   21.5, w_t:  0.00
For training: accuracy-8%; IOU-40%
For validation: accuracy-76%; IOU-54%
Epoch:    2 - Loss:  35.76, loss_sup:   35.8, loss_unsup:   18.3, w_t:  0.00
Epoch:    3 - Loss:  34.11, loss_sup:   34.1, loss_unsup:   16.9, w_t:  0.00
Epoch:    4 - Loss:  32.33, loss_sup:   32.3, loss_unsup:   15.9, w_t:  0.00
Epoch:    5 - Loss:  31.84, loss_sup:   31.8, loss_unsup:   15.9, w_t:  0.00
Epoch:    6 - Loss:  31.72, loss_sup:   31.7, loss_unsup:   16.6, w_t:  0.00
For training: accuracy-8%; IOU-38%
For validation: accuracy-79%; IOU-59%
Epoch:    7 - Loss:  31.35, loss_sup:   31.2, loss_unsup:   15.9, w_t:  0.01
Epoch:    8 - Loss:  30.50, loss_sup:   30.3, loss_unsup:   15.6, w_t:  0.01
Epoch:    9 - Loss:  30.77, loss_sup:   30.4, loss_unsup:   15.4, w_t:  0.02
Epoch:   10 - Loss:  30.19, loss_sup:   29.7, loss_unsup:   15.5, w_t:  0.03
Epoch:   11 - Loss:  29.36, loss_sup:   28.7, loss_unsup:   14.8, w_t:  0.04
For tra